In [ ]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sqlalchemy import text
import pandas as pd
import os

load_dotenv()

DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
PATH_TO_CERT = os.getenv('PATH_TO_CERT')

connection_str = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

engine = create_engine(
    connection_str,
    connect_args={
        "sslmode": "verify-full",
        "sslrootcert": PATH_TO_CERT,
        "target_session_attrs": "read-write"
    }
)

In [12]:
create_table_query = """
drop table if exists posts;
"""

try:
    with engine.begin() as conn:
        conn.execute(text(create_table_query))
except Exception as e:
    print("Ошибка:", e)


In [4]:
with engine.connect() as conn:
    df = pd.read_sql('''
select * from posts
    ''', conn)

df

,id,text,created_at


In [15]:
rbc = pd.read_csv("src/dataset/rbc/channel_rbc_news_posts.csv")
rbc["message_dt"] = pd.to_datetime(rbc["message_dt"]).dt.date
rbc = rbc.drop(columns=["original_author"])
rbc = rbc[["message_id", "channel_id", "message_dt", "views", "content"]]

In [16]:
rbc

,message_id,channel_id,message_dt,views,content
0,137230,rbc_news,2025-12-03,15453.0,Отправить деньги за рубеж и помочь близким — т...
1,137228,rbc_news,2025-12-03,40045.0,Суд признал писателя Бориса Акунина (настоящее...
2,137226,rbc_news,2025-12-03,53463.0,"На характере переговоров с США, прошедших нака..."
3,137224,rbc_news,2025-12-03,56667.0,Еврокомиссия намерена запретить исполнение вну...
4,137223,rbc_news,2025-12-03,55604.0,🎙 Прямо сейчас в эфире Радио РБК обсуждаем пла...
...,...,...,...,...,...
4842,116045,rbc_news,2025-04-15,108803.0,Суд на Сахалине вынес первое решение по делу о...
4843,116044,rbc_news,2025-04-15,108074.0,Оператор национальных лотерей Франции стал отв...
4844,116043,rbc_news,2025-04-15,143013.0,🐘 Слоны во время землетрясения в зоопарке Сан-...
4845,116041,rbc_news,2025-04-15,125020.0,Аэропорт южнокорейского города Муан регулярно ...


In [17]:
rbc.to_sql('posts', engine, if_exists='replace', index=False)

847

In [18]:
with engine.connect() as conn:
    df = pd.read_sql('''
select * from posts
    ''', conn)

df

,message_id,channel_id,message_dt,views,content
0,137230,rbc_news,2025-12-03,15453.0,Отправить деньги за рубеж и помочь близким — т...
1,137228,rbc_news,2025-12-03,40045.0,Суд признал писателя Бориса Акунина (настоящее...
2,137226,rbc_news,2025-12-03,53463.0,"На характере переговоров с США, прошедших нака..."
3,137224,rbc_news,2025-12-03,56667.0,Еврокомиссия намерена запретить исполнение вну...
4,137223,rbc_news,2025-12-03,55604.0,🎙 Прямо сейчас в эфире Радио РБК обсуждаем пла...
...,...,...,...,...,...
4842,116045,rbc_news,2025-04-15,108803.0,Суд на Сахалине вынес первое решение по делу о...
4843,116044,rbc_news,2025-04-15,108074.0,Оператор национальных лотерей Франции стал отв...
4844,116043,rbc_news,2025-04-15,143013.0,🐘 Слоны во время землетрясения в зоопарке Сан-...
4845,116041,rbc_news,2025-04-15,125020.0,Аэропорт южнокорейского города Муан регулярно ...


In [40]:
from qdrant_client import QdrantClient, models
from dotenv import load_dotenv
import numpy as np
import os

load_dotenv()

QDRANT_URL = os.getenv('QDRANT_URL')

client = QdrantClient(
    url=QDRANT_URL,
)

In [2]:
client.create_collection(
    collection_name="my_collection",
    vectors_config=models.VectorParams(
        size=384, 
        distance=models.Distance.COSINE
    )
)


True

In [ ]:
points = [
    models.PointStruct(
        id=1,
        vector=np.ones(384).tolist(),
        payload={"text": "Пример документа 1"}
    ),
    models.PointStruct(
        id=2,
        vector=(-np.ones(384)).tolist(),
        payload={"text": "Пример документа 2"}
    )
]

client.upsert(
    collection_name="my_collection",
    points=points,
    wait=True
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [42]:
results = client.query_points(
    collection_name="my_collection",
    query=(-np.ones(384)).tolist(),
    limit=1
)

In [43]:
results.points[0]

ScoredPoint(id=2, version=3, score=0.9999998, payload={'text': 'Пример документа 2'}, vector=None, shard_key=None, order_value=None)